🤓 Harbey Perdomo Montoya  
C.C. 1.072.961.355  
**Maestria en analitica de datos**

# Taller_1 Tokenización de certificado de camara de Comercio

In [1]:
#@title  Instalar librerias
!pip install spacy
!python -m spacy download es_core_news_md # Modelo Medio
!pip install pdfplumber --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 13.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
#@title  Librerías para el desarrollo del ejercicio
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [2]:
#@title Conección con Colap
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#@title Cargar el modelo en español
nlp = spacy.load("es_core_news_md") # se cambio al modelo mediano para comparar resultados cn el pequeño

In [4]:
#@title Selección de certificado a Tokenizar
pdf_path = "/content/drive/MyDrive/Maestria en analítica estratégica de datos/4. Semestrre/Analisis_textos/Taller_1/certificado (2).pdf"

In [5]:
#@title Identificación del encabezada para ser removido de la tokenización
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"


Mejora la precisión de la extracción de palabras del código proporcionado para texto en español.

In [6]:
#@title Funsión para la extración de textos del certificado
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página
    watermark_pattern = r"MATRICULA NO RENOVADA Actualice su registro y evite sanciones" # Patrón de la marca de agua

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex

                # Unir palabras guionadas al final de línea
                text = re.sub(r"-\n", "", text)
                text = re.sub(r"\n", " ", text) # Reemplazar saltos de línea restantes por espacios




                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [7]:
#@title Funsión para Normalizar y convertir los tokens en palabras

def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    # Eliminar caracteres especiales y puntuación, excepto espacios
    text = re.sub(r"[^\w\s]", "", text)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    # Modificar la condición para incluir tokens que son alfabéticos o dígitos
    tokens = [token.text for token in doc if token.is_alpha or token.is_digit]
    return tokens

In [8]:
#@title Ver los resultados de Tokenización con SpaCy
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'el', 'contenido', 'y', 'confiabilidad', 'de', 'este', 'certificado', 'ingresando', 'a', 'wwwcamarabaqorgco', 'y', 'digite', 'el', 'codigo', 'para', 'que', 'visualice', 'la', 'imagen', 'generada', 'al', 'momento', 'de', 'su', 'expedicion', 'la', 'verificacion', 'se', 'puede', 'realizar', 'de', 'manera', 'ilimitada', 'durante', '60', 'dias', 'calendario', 'contados', 'a', 'partir', 'de', 'la', 'fecha', 'de', 'su', 'expedicion', 'atencion', 'este', 'comerciante']


In [9]:
#@title Guardar los tokens preprocesados en un archivo de texto.

with open("/content/drive/MyDrive/Maestria en analítica estratégica de datos/4. Semestrre/Analisis_textos/Taller_1/certificado_limpio_2.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))